Test CNN on spectrogram

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
import time
import numpy as np
import torch.utils.data as data_utils
import torch.nn.functional as F

from scipy import signal
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

Organize data into something useable

In [12]:
def split_train_test_kfold(feature_filename, target_filename, k_fold=1):
    features_og = np.load(feature_filename)
    targets_og = np.load(target_filename)
    print(sum(targets_og),targets_og.shape)

    # Split data into train and test
    split = np.arange(len(targets_og))
    np.random.shuffle(split)
    features_og = features_og[split]
    targets_og = targets_og[split]
    features_og_0 = features_og[np.where(targets_og==0)[0]]
    features_og_1 = features_og[np.where(targets_og==1)[0]]
    targets_og_0 = targets_og[np.where(targets_og==0)[0]]
    targets_og_1 = targets_og[np.where(targets_og==1)[0]]
    N_0 = len(targets_og_0)
    N_1 = len(targets_og_1)
    
    features_og_train = np.vstack([features_og_0,features_og_1])
    targets_og_train = np.vstack([targets_og_0,targets_og_1])
    sample_list = []
    
    num_seizure_per = float(N_1)//k_fold
    num_remainder = float(N_1) % k_fold
    for i in range(k_fold):
        if (i+1)==k_fold:
            temp1 = np.arange(i*N_0//k_fold,N_0, dtype=np.int64)
            temp2 = np.arange(i*N_1//k_fold,N_1, dtype=np.int64)+N_0
        else:
            temp1 = np.arange(i*N_0//k_fold,(i+1)*N_0//k_fold, dtype=np.int64)
            temp2 = np.arange(i*N_1//k_fold,(i+1)*N_1//k_fold, dtype=np.int64)+N_0
        temp2 = np.repeat(temp2,250)
        sample_list.append(np.hstack([temp1,temp2]))
    
    train_sampler_list = []
    test_sampler_list = []
    for i in range(k_fold):
        temp = np.where(np.arange(len(sample_list), dtype=np.int64) != i)[0]
        train =  np.hstack([sample_list[x] for x in temp])
        test = sample_list[i]
        train_sampler_list.append(SubsetRandomSampler(train))
        test_sampler_list.append(SubsetRandomSampler(test))
        
    # Convert data to tensor dataset
    features = torch.from_numpy(features_og_train).float()
    targets = torch.from_numpy(targets_og_train).long()
    targets = torch.squeeze(targets)
    train = data_utils.TensorDataset(features, targets)
    
    return train_sampler_list, test_sampler_list, train
    
def split_train_test(train_percent, k_fold=0):
    features_og = np.load('features_nonsliding_ch.npy')
    targets_og = np.load('targets_nonsliding_ch.npy')
    print(sum(targets_og),targets_og.shape)

    # Split data into train and test
    split = np.arange(len(targets_og))
    np.random.shuffle(split)
    features_og = features_og[split]
    targets_og = targets_og[split]
    features_og_0 = features_og[np.where(targets_og==0)[0]]
    features_og_1 = features_og[np.where(targets_og==1)[0]]
    targets_og_0 = targets_og[np.where(targets_og==0)[0]]
    targets_og_1 = targets_og[np.where(targets_og==1)[0]]
    N_0 = len(targets_og_0)
    N_1 = len(targets_og_1)
    
    features_og_train = np.vstack([features_og_0[:int(train_percent*N_0)],features_og_1[:int(train_percent*N_1)]])
    targets_og_train = np.vstack([targets_og_0[:int(train_percent*N_0)],targets_og_1[:int(train_percent*N_1)]])
    features_og_test = np.vstack([features_og_0[int(train_percent*N_0):],features_og_1[int(train_percent*N_1):]])
    targets_og_test = np.vstack([targets_og_0[int(train_percent*N_0):],targets_og_1[int(train_percent*N_1):]])
    print(sum(targets_og_train), sum(targets_og_test))
    

    # Balance dataset
    # ~1/4000 seizure events
    idx = np.hstack([np.where(targets_og_train == 0)[0], 
                     np.repeat(np.where(targets_og_train == 1)[0], 100)]) # Oversample
    features = features_og_train[idx]
    targets = targets_og_train[idx]

    # Convert data to tensor dataset
    features = torch.from_numpy(features).float()
    targets = torch.from_numpy(targets).long()
    targets = torch.squeeze(targets)
    train = data_utils.TensorDataset(features, targets)

    N = features.size()[0]
    sample_list = np.arange(N, dtype=np.int64)
    np.random.shuffle(sample_list)
    percent_train = 1.0

    #Training
    n_training_samples = int(N*percent_train)
    train_sampler = SubsetRandomSampler(sample_list[:n_training_samples])

    #Validation
    val_sampler = SubsetRandomSampler(sample_list[:n_training_samples])

    #Test data
    features = torch.from_numpy(features_og_test).float()
    targets = torch.from_numpy(targets_og_test).long()
    targets = torch.squeeze(targets)
    test = data_utils.TensorDataset(features, targets)
    return train, test, train_sampler, val_sampler

In [4]:
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 1, output channels = 18
        self.conv1 = torch.nn.Conv2d(23, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 16 *16)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [5]:
#DataLoader takes in a dataset and a sampler for loading (num_workers deals with system level memory) 
def get_train_loader(batch_size, train_sampler):
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=2)
    return(train_loader)

#Test and validation loaders have constant batch sizes, so we can define them directly

def trainNet(net, train_sampler, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size, train_sampler)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0.0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data[0]
            total_train_loss += loss_size.data[0]
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.6f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data[0]
            
        print("Validation loss = {:.2f}".format(total_val_loss / max(1,len(val_loader))))
        

        print("Training finished, took {:.2f}s".format(time.time() - training_start_time))
        
def calculate_cm(test_sampler):
    test_loader = torch.utils.data.DataLoader(train, sampler=test_sampler, batch_size=2)
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data          
            outputs = CNN(images)
            _, predicted = torch.max(outputs.data, 1)
            for x,y in zip(predicted, labels):
                y_true.append(int(y.numpy()))
                y_pred.append(int(x.numpy()))
    return confusion_matrix(np.array(y_true), np.array(y_pred))

In [18]:
# cross validation
k_fold = 4
batch_size = 32
feature_filename = 'features_nonsliding_ch.npy' #stft vs spect
target_filename = 'targets_nonsliding_ch.npy'

train_sampler_list, test_sampler_list, train = split_train_test_kfold(feature_filename, target_filename, k_fold=k_fold)

cm = np.zeros((2,2))
for train_sampler, test_sampler in zip(train_sampler_list, test_sampler_list):
    CNN = SimpleCNN()
    val_loader = torch.utils.data.DataLoader(train, batch_size=32, sampler=train_sampler, num_workers=2)
    trainNet(CNN, train_sampler, batch_size=batch_size, n_epochs=10, learning_rate=1e-6)# 2,5,1e-6 
    cm += calculate_cm(test_sampler)
    print(cm)

tn, fp, fn, tp = cm.ravel()
fn=fn/250
tp=tp/250
acc = (tp+tn)/(tn+fp+fn+tp)
sen = tp/(tp+fn)
spec = tn/(tn+fp)
prec = tp/(tp+fp)
F1 = 2 * (prec * sen) / (prec + sen)

print(cm)
print('accuracy: ',acc)
print('sensitivity:', sen)
print('specificity:', spec)
print('precision:', prec)
print('F1:', F1)

[ 38.] (10379, 1)
(10379, 23, 64, 16)
(10379, 1)
===== HYPERPARAMETERS =====
batch_size= 32
epochs= 10
learning_rate= 1e-06


/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:55: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:56: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 1, 10% 	 train_loss: 4.211040 took: 1.45s
Epoch 1, 20% 	 train_loss: 2.013771 took: 0.61s
Epoch 1, 30% 	 train_loss: 1.198637 took: 0.61s
Epoch 1, 40% 	 train_loss: 0.809367 took: 0.63s
Epoch 1, 50% 	 train_loss: 0.636083 took: 0.61s
Epoch 1, 60% 	 train_loss: 0.504578 took: 0.61s
Epoch 1, 70% 	 train_loss: 0.462832 took: 0.64s
Epoch 1, 80% 	 train_loss: 0.400315 took: 0.62s
Epoch 1, 90% 	 train_loss: 0.370664 took: 0.61s


/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:75: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Validation loss = 0.31
Training finished, took 11.34s
Epoch 2, 10% 	 train_loss: 0.298562 took: 1.46s
Epoch 2, 20% 	 train_loss: 0.277432 took: 0.60s
Epoch 2, 30% 	 train_loss: 0.241240 took: 0.61s
Epoch 2, 40% 	 train_loss: 0.215489 took: 0.61s
Epoch 2, 50% 	 train_loss: 0.199394 took: 0.64s
Epoch 2, 60% 	 train_loss: 0.186847 took: 0.61s
Epoch 2, 70% 	 train_loss: 0.163653 took: 0.61s
Epoch 2, 80% 	 train_loss: 0.161275 took: 0.61s
Epoch 2, 90% 	 train_loss: 0.155828 took: 0.61s
Validation loss = 0.13
Training finished, took 22.64s
Epoch 3, 10% 	 train_loss: 0.123987 took: 1.43s
Epoch 3, 20% 	 train_loss: 0.122875 took: 0.61s
Epoch 3, 30% 	 train_loss: 0.113584 took: 0.62s
Epoch 3, 40% 	 train_loss: 0.113597 took: 0.61s
Epoch 3, 50% 	 train_loss: 0.107041 took: 0.61s
Epoch 3, 60% 	 train_loss: 0.098871 took: 0.62s
Epoch 3, 70% 	 train_loss: 0.088949 took: 0.61s
Epoch 3, 80% 	 train_loss: 0.084781 took: 0.61s
Epoch 3, 90% 	 train_loss: 0.084938 took: 0.61s
Validation loss = 0.08
Train

Process Process-1347:
Process Process-1348:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/Christian/

KeyboardInterrupt: 

In [ ]:
tn, fp, fn, tp = cm.ravel()
fn=fn/250
tp=tp/250
acc = (tp+tn)/(tn+fp+fn+tp)
sen = tp/(tp+fn)
spec = tn/(tn+fp)
prec = tp/(tp+fp)
F1 = 2 * (prec * sen) / (prec + sen)

print(cm)
print('accuracy: ',acc)
print('sensitivity:', sen)
print('specificity:', spec)
print('precision:', prec)
print('F1:', F1)

In [ ]:
cm = confusion_matrix(np.array(y_true), np.array(y_pred))
    
test_loader = torch.utils.data.DataLoader(test, batch_size=2)
cm = np.zeros((2,2))
total=0
correct=0
y_true = []
y_pred = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data          
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for x,y in zip(predicted, labels):
            y_true.append(int(y.numpy()))
            y_pred.append(int(x.numpy()))
            
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

cm = confusion_matrix(np.array(y_true), np.array(y_pred))
tn, fp, fn, tp = cm.ravel()
acc = (tp+tn)/(tn+fp+fn+tp)
sen = tp/(tp+fn)
spec = tn/(tn+fp)
prec = tp/(tp+fp)
F1 = 2 * (prec * sen) / (prec + sen)

print(cm)
print('accuracy: ',acc)
print('sensitivity:', sen)
print('specificity:', spec)
print('precision:', prec)
print('F1:', F1)

### Visualize CNN running on an actual file

In [ ]:
df = {}
df['1'] = a # Figure out how to do python 2 wrapp
width = 8

feature, target = df_to_spectrogram_FT(df, sliding=True, avg=True, sliding_ft=False, width=8, stft=True)

# Convert feature to tensor
# Run it into the CNN 

### Visualize some example of CNN labeling

In [ ]:
limit = 10e10
for i, data in enumerate(test_loader):
    if i > limit:
        break
    images, labels = data
    for x,y in zip(images,labels):
        x = x.numpy()
        y = y.numpy()
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.numpy()[0]
        if y==1:
            x=np.mean(x,axis=0)

            plt.pcolormesh(np.arange(x.shape[1])*0.875,np.arange(x.shape[0]),x)
            plt.ylabel('Frequency [Hz]')
            plt.xlabel('Time [sec]')
            plt.colorbar()
            if y:
                plt.title('Seizure Spectrogram (dB)')
            else:
                plt.title('Normal Spectrogram (dB)')
            plt.show()